In [1]:
import copy
import itertools
import logging

In [2]:
class LoggerMixin(object):
    logger_level = logging.INFO

    @property
    def logger(self):
        logger = logging.getLogger(self.__class__.__name__)
        if (logger.hasHandlers()):
            logger.handlers.clear()
        handler = logging.StreamHandler()
        formatter = logging.Formatter('%(name)s - %(levelname)s \n%(message)s')
        handler.setFormatter(formatter)
        logger.addHandler(handler)
        logger.setLevel(self.logger_level)
        return logger


In [3]:
class Matrix(LoggerMixin, object):
    def __init__(self, data):
        n = len(data)  # number of rows
        m = len(data[0])  # number of columns
        self.shape = (n, m)
        self.data = [[float(element) for element in row] for row in data]

    @classmethod
    def zeros(cls, shape):
        n, m = shape
        matrix = cls([
            [0 for k in range(m)]
            for j in range(n)
        ])
        return matrix

    @classmethod
    def identity(cls, n):
        matrix = cls.zeros((n, n))
        for j in range(n): matrix[j][j] = 1.0
        return matrix

    def is_square(self):
        return self.shape[0] == self.shape[1]

    def copy(self):
        return Matrix(copy.deepcopy(self.data))

    def __len__(self):
        return self.n

    def __repr__(self):
        return "Matrix([\n    " \
            + ",\n    ".join(repr(row) for row in self.data) + "\n])"

    def __getitem__(self, item):
        return self.data[item]

    def __add__(self, other):
        result = Matrix.zeros(self.shape)
        for j, k in itertools.product(range(result.shape[0]), range(result.shape[1])):
            try:
                assert self.shape == other.shape
                result[j][k] = self[j][k] + other[j][k]
            except AttributeError:
                result[j][k] = self[j][k] + other
        return result

    def __sub__(self, other):
        result = Matrix.zeros(self.shape)
        for j, k in itertools.product(range(result.shape[0]), range(result.shape[1])):
            try:
                assert self.shape == other.shape
                result[j][k] = self[j][k] - other[j][k]
            except AttributeError:
                result[j][k] = self[j][k] - other
        return result

    def __mul__(self, other):
        result = self.copy()
        for j, k in itertools.product(range(result.shape[0]), range(result.shape[1])):
            try:
                assert self.shape == other.shape
                result[j][k] *= other[j][k]
            except AttributeError:
                result[j][k] *= other
        return result
    
    def __matmul__(self, other):
        return self.multiply_by(other)

    def add(self, other):
        assert self.shape == other.shape
        new = self.copy()
        for j, k in itertools.product(range(self.shape[0]), range(self.shape[1])):
            new[j][k] += other[j][k]
        return new

    def multiply_by(self, other):
        assert self.shape[1] == other.shape[0]
        return Matrix([
            [
                float(sum(
                    self.data[self_row][self_column] \
                        * other.data[other_row][other_column]
                    for self_column, other_row in 
                        zip(range(self.shape[1]), range(other.shape[0]))
                ))
                for other_column in range(other.shape[1])
            ]
            for self_row in range(self.shape[0])
        ])

    def transpose(self):
        new = Matrix.zeros((self.shape[1], self.shape[0]))
        for j, k in itertools.product(range(self.shape[0]), range(self.shape[1])):
            new[k][j] = self[j][k]
        return new

    def invert(self):
        n = self.shape[0]
        operations_so_far = Matrix.identity(n)
        self_copy = self.copy()

        for k in range(n):
            self.logger.debug(f"processing column {k}")
            self.logger.debug("self_copy = " + str(self_copy))

            # reorder rows
            row_reordering = \
                list(range(k)) + \
                list(reversed(sorted(
                    range(k, n),
                    key=lambda j: abs(self_copy[j][k])
            )))
            reorder_op = Matrix([Matrix.identity(n).data[j] for j in row_reordering])
            self.logger.debug("reorder_op = " + str(reorder_op))
            operations_so_far = reorder_op.multiply_by(operations_so_far)
            self_copy = reorder_op.multiply_by(self_copy)
            self.logger.debug("self_copy = " + str(self_copy))

            # normalize this row
            normalization_op = Matrix.identity(n)
            normalization_op[k][k] = 1.0 / self_copy[k][k]
            self.logger.debug("normalization_op = " + str(normalization_op))
            operations_so_far = normalization_op.multiply_by(operations_so_far)
            self_copy = normalization_op.multiply_by(self_copy)
            self.logger.debug("self_copy = " + str(self_copy))

            # eliminate the other rows
            eliminations_op = Matrix.identity(n)
            for j in range(n):
                if j == k:
                    continue
                eliminations_op[j][k] = -1.0 * self_copy[j][k]
            self.logger.debug("eliminations_op = " + str(eliminations_op))
            operations_so_far = eliminations_op.multiply_by(operations_so_far)
            self.logger.debug("operations_so_far = " + str(operations_so_far))
            self_copy = eliminations_op.multiply_by(self_copy)
            self.logger.debug("self_copy = " + str(self_copy))

            self.logger.debug(f"done with column {k}\n")
            
        return operations_so_far


# Basics

In [4]:
X = Matrix([[1, 2], [3, 4], [5, 6]])
X

Matrix([
    [1.0, 2.0],
    [3.0, 4.0],
    [5.0, 6.0]
])

In [5]:
Y = Matrix([[1, 2, 3], [4, 5, 6]])
Y

Matrix([
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0]
])

In [6]:
Matrix.zeros((2, 3))

Matrix([
    [0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0]
])

In [7]:
Matrix.identity(8)

Matrix([
    [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
])

# Matrix addition

In [8]:
X = Matrix([
    [1, 3]
])
Y = Matrix([
    [5, 11]
])

In [9]:
X.add(Y)

Matrix([
    [6.0, 14.0]
])

In [10]:
Y.add(X)

Matrix([
    [6.0, 14.0]
])

# Matrix multiplication

In [11]:
Matrix([
    [2]
]).multiply_by(Matrix([
    [3]
]))

Matrix([
    [6.0]
])

In [12]:
Matrix([
    [1, 2],
    [3, 4]
]).multiply_by(Matrix([
    [1, 0],
    [0, 1]
]))

Matrix([
    [1.0, 2.0],
    [3.0, 4.0]
])

In [13]:
Matrix([
    [1, 2],
    [3, 4]
]).multiply_by(Matrix([
    [0, 1],
    [1, 0]
]))

Matrix([
    [2.0, 1.0],
    [4.0, 3.0]
])

In [14]:
Matrix([
    [1, 2],
    [3, 4]
]).multiply_by(Matrix([
    [2, 0],
    [0, 2]
]))

Matrix([
    [2.0, 4.0],
    [6.0, 8.0]
])

In [15]:
Matrix([
    [1, 2],
    [3, 4]
]).multiply_by(Matrix([
    [1, 0],
    [0, 2]
]))

Matrix([
    [1.0, 4.0],
    [3.0, 8.0]
])

In [16]:
Matrix([
    [1, 2],
    [3, 4]
]).multiply_by(Matrix([
    [1, 1],
    [0, 1]
]))

Matrix([
    [1.0, 3.0],
    [3.0, 7.0]
])

# Matrix transpose

In [17]:
X = Matrix([
    [1]
])
print(X)
print(X.shape)
X_t = X.transpose()
print(X_t)
print(X_t.shape)


Matrix([
    [1.0]
])
(1, 1)
Matrix([
    [1.0]
])
(1, 1)


In [18]:
X = Matrix([
    [1, 0]
])
print(X)
print(X.shape)
X_t = X.transpose()
print(X_t)
print(X_t.shape)


Matrix([
    [1.0, 0.0]
])
(1, 2)
Matrix([
    [1.0],
    [0.0]
])
(2, 1)


In [19]:
X = Matrix([
    [1, 2, 3],
    [4, 5, 6]
])
print(X)
print(X.shape)
X_t = X.transpose()
print(X_t)
print(X_t.shape)


Matrix([
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0]
])
(2, 3)
Matrix([
    [1.0, 4.0],
    [2.0, 5.0],
    [3.0, 6.0]
])
(3, 2)


# Matrix inversion

In [20]:
X = Matrix([
    [1, 0],
    [0, 1]
])
X_inv = X.invert()

print("X =", X)
print("X_inv =", X_inv)
print("X_inv X =", X_inv.multiply_by(X))
print("X X_inv =", X.multiply_by(X_inv))

X = Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])
X_inv = Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])
X_inv X = Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])
X X_inv = Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])


In [21]:
X = Matrix([
    [1, 0],
    [0, 2]
])
X_inv = X.invert()

print("X =", X)
print("X_inv =", X_inv)
print("X_inv X =", X_inv.multiply_by(X))
print("X X_inv =", X.multiply_by(X_inv))

X = Matrix([
    [1.0, 0.0],
    [0.0, 2.0]
])
X_inv = Matrix([
    [1.0, 0.0],
    [0.0, 0.5]
])
X_inv X = Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])
X X_inv = Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])


In [22]:
X = Matrix([
    [2, 0],
    [0, 1]
])
X_inv = X.invert()

print("X =", X)
print("X_inv =", X_inv)
print("X_inv X =", X_inv.multiply_by(X))
print("X X_inv =", X.multiply_by(X_inv))

X = Matrix([
    [2.0, 0.0],
    [0.0, 1.0]
])
X_inv = Matrix([
    [0.5, 0.0],
    [0.0, 1.0]
])
X_inv X = Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])
X X_inv = Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])


In [23]:
X = Matrix([
    [1, 0, 0],
    [0, 2, 0],
    [0, 0, 3]
])
X_inv = X.invert()

print("X =", X)
print("X_inv =", X_inv)
print("X_inv X =", X_inv.multiply_by(X))
print("X X_inv =", X.multiply_by(X_inv))

X = Matrix([
    [1.0, 0.0, 0.0],
    [0.0, 2.0, 0.0],
    [0.0, 0.0, 3.0]
])
X_inv = Matrix([
    [1.0, 0.0, 0.0],
    [0.0, 0.5, 0.0],
    [0.0, 0.0, 0.3333333333333333]
])
X_inv X = Matrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0]
])
X X_inv = Matrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0]
])


In [24]:
X = Matrix([
    [1, 0, 0],
    [0, 2, 0],
    [0, 0, 1]
])
X_inv = X.invert()

print("X =", X)
print("X_inv =", X_inv)
print("X_inv X =", X_inv.multiply_by(X))
print("X X_inv =", X.multiply_by(X_inv))

X = Matrix([
    [1.0, 0.0, 0.0],
    [0.0, 2.0, 0.0],
    [0.0, 0.0, 1.0]
])
X_inv = Matrix([
    [1.0, 0.0, 0.0],
    [0.0, 0.5, 0.0],
    [0.0, 0.0, 1.0]
])
X_inv X = Matrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0]
])
X X_inv = Matrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0]
])


In [25]:
X = Matrix([
    [3, 1],
    [4, 2]
])
X.logger_level = logging.DEBUG

X_inv = X.invert()

print("X =", X)
print("X_inv =", X_inv)
print("X_inv X =", X_inv.multiply_by(X))
print("X X_inv =", X.multiply_by(X_inv))

Matrix - DEBUG 
processing column 0
Matrix - DEBUG 
self_copy = Matrix([
    [3.0, 1.0],
    [4.0, 2.0]
])
Matrix - DEBUG 
reorder_op = Matrix([
    [0.0, 1.0],
    [1.0, 0.0]
])
Matrix - DEBUG 
self_copy = Matrix([
    [4.0, 2.0],
    [3.0, 1.0]
])
Matrix - DEBUG 
normalization_op = Matrix([
    [0.25, 0.0],
    [0.0, 1.0]
])
Matrix - DEBUG 
self_copy = Matrix([
    [1.0, 0.5],
    [3.0, 1.0]
])
Matrix - DEBUG 
eliminations_op = Matrix([
    [1.0, 0.0],
    [-3.0, 1.0]
])
Matrix - DEBUG 
operations_so_far = Matrix([
    [0.0, 0.25],
    [1.0, -0.75]
])
Matrix - DEBUG 
self_copy = Matrix([
    [1.0, 0.5],
    [0.0, -0.5]
])
Matrix - DEBUG 
done with column 0

Matrix - DEBUG 
processing column 1
Matrix - DEBUG 
self_copy = Matrix([
    [1.0, 0.5],
    [0.0, -0.5]
])
Matrix - DEBUG 
reorder_op = Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])
Matrix - DEBUG 
self_copy = Matrix([
    [1.0, 0.5],
    [0.0, -0.5]
])
Matrix - DEBUG 
normalization_op = Matrix([
    [1.0, 0.0],
    [0.0, -2.0]
])
M

X = Matrix([
    [3.0, 1.0],
    [4.0, 2.0]
])
X_inv = Matrix([
    [1.0, -0.5],
    [-2.0, 1.5]
])
X_inv X = Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])
X X_inv = Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])


In [26]:
Matrix([
    [1.0, -0.5],
    [-2.0, 1.5]
]).multiply_by(X)

Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])

In [27]:
X = Matrix([
    [3, 1],
    [4, 2]
])
X_inv = X.invert()

print("X =", X)
print("X_inv =", X_inv)
print("X_inv X =", X_inv.multiply_by(X))
print("X X_inv =", X.multiply_by(X_inv))

X = Matrix([
    [3.0, 1.0],
    [4.0, 2.0]
])
X_inv = Matrix([
    [1.0, -0.5],
    [-2.0, 1.5]
])
X_inv X = Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])
X X_inv = Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])


In [28]:
X = Matrix([
    [2, 5, 0],
    [1, 0, 1],
    [1, 2, 0],
])
X_inv = X.invert()

print(X)
print(X_inv)
print(X.multiply_by(X_inv))
print(X_inv.multiply_by(X))

Matrix([
    [2.0, 5.0, 0.0],
    [1.0, 0.0, 1.0],
    [1.0, 2.0, 0.0]
])
Matrix([
    [-2.0, 0.0, 5.0],
    [1.0, 0.0, -2.0],
    [2.0, 1.0, -5.0]
])
Matrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0]
])
Matrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0]
])


# nice syntax

In [29]:
X = Matrix([
    [3, 1],
    [4, 2]
])
X_inv = X.invert()
print("X =", X)
print("X_inv =", X_inv)

X = Matrix([
    [3.0, 1.0],
    [4.0, 2.0]
])
X_inv = Matrix([
    [1.0, -0.5],
    [-2.0, 1.5]
])


### verify: X_inv is the inverse

In [30]:
X_inv.multiply_by(X)

Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])

In [31]:
X.multiply_by(X_inv)

Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])

### demo cute operations

In [32]:
X + X_inv

Matrix([
    [4.0, 0.5],
    [2.0, 3.5]
])

In [33]:
X - X_inv

Matrix([
    [2.0, 1.5],
    [6.0, 0.5]
])

In [34]:
X * X_inv

Matrix([
    [3.0, -0.5],
    [-8.0, 3.0]
])

In [35]:
X @ X_inv

Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])

In [36]:
X_inv @ X

Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])

### with another matrix

In [37]:
X = Matrix([
    [2, 5, 0],
    [1, 0, 1],
    [1, 2, 0],
])
X_inv = X.invert()

print(X)
print(X_inv)
print(X @ X_inv)
print(X_inv @ X)

Matrix([
    [2.0, 5.0, 0.0],
    [1.0, 0.0, 1.0],
    [1.0, 2.0, 0.0]
])
Matrix([
    [-2.0, 0.0, 5.0],
    [1.0, 0.0, -2.0],
    [2.0, 1.0, -5.0]
])
Matrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0]
])
Matrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0]
])


# Linear regression

In [38]:
y = Matrix([
    [1], 
    [2],
    [3]
])
X = Matrix([
    [1, 2],
    [5, -1],
    [9, 0.5]
])

print("y =", y)
print("X =", X)

y = Matrix([
    [1.0],
    [2.0],
    [3.0]
])
X = Matrix([
    [1.0, 2.0],
    [5.0, -1.0],
    [9.0, 0.5]
])


In [39]:
beta_hat = (X.transpose().multiply_by(X)).invert().multiply_by(X.transpose()).multiply_by(y)
beta_hat

Matrix([
    [0.35254691689008044],
    [0.18498659517426275]
])

In [40]:
beta_hat = (X.transpose() @ X).invert() @ X.transpose() @ y
beta_hat

Matrix([
    [0.35254691689008044],
    [0.18498659517426275]
])

In [41]:
# check...
import sklearn as skl
from sklearn.linear_model import LinearRegression

In [42]:
lr = LinearRegression(fit_intercept=False)
lr.fit(X.data, y.data)
lr.coef_

array([[0.35254692, 0.1849866 ]])

In [43]:
import statsmodels.api as sm
import numpy as np

In [44]:
X_np = np.array(X.data)
X_np, X_np.shape

(array([[ 1. ,  2. ],
        [ 5. , -1. ],
        [ 9. ,  0.5]]), (3, 2))

In [45]:
y_np = np.array(y.data)
y_np, y_np.shape

(array([[1.],
        [2.],
        [3.]]), (3, 1))

In [46]:
model = sm.OLS(y_np, X_np)

In [47]:
results = model.fit()

In [48]:
results.summary()

/Users/william/.local/share/virtualenvs/ml-study-group-ExtSe04q/lib/python3.7/site-packages/statsmodels/stats/stattools.py:71: ValueWarning: omni_normtest is not valid with less than 8 observations; 3 samples were given.
  "samples were given." % int(n), ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.977
Model:                            OLS   Adj. R-squared (uncentered):              0.930
Method:                 Least Squares   F-statistic:                              20.99
Date:                Wed, 09 Oct 2019   Prob (F-statistic):                       0.153
Time:                        16:54:14   Log-Likelihood:                        -0.92640
No. Observations:                   3   AIC:                                      5.853
Df Residuals:                       1   BIC:                                      4.050
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.3525      0.055      6.377      0.099      -0.350       1.055
x2             0.1850      0.250      0.741      0.594      -2.986       3.356
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   1.516
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.451
Skew:                          -0.582   Prob(JB):                        0.798
Kurtosis:                       1.500   Cond. No.                         4.52
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## covariance of beta_hat

Var(beta_hat) = (X.transpose().multiply_by(X)).inverse().multiply_by(sigma_hat_squared)
sigma_hat_squared = residuals.transpose().multiply_by(residuals)
residuals = y - X.multiply_by(beta_hat)

In [49]:
# (X.transpose().multiply_by(X).invert()) * (y - X.multiply_by(beta_hat)).transpose()
# lol, screw this

In [50]:
y_hat = X @ beta_hat
y_hat

Matrix([
    [0.7225201072386059],
    [1.5777479892761397],
    [3.2654155495978556]
])

In [51]:
residual = y - y_hat
residual

Matrix([
    [0.27747989276139406],
    [0.4222520107238603],
    [-0.2654155495978556]
])

In [52]:
residual_sum_of_squares = residual.transpose() @ residual
residual_sum_of_squares

Matrix([
    [0.32573726541554954]
])

In [53]:
cov_beta_hat = (X.transpose() @ X).invert() * residual_sum_of_squares.data[0][0]

In [54]:
cov_beta_hat

Matrix([
    [0.0030565158953201697, -0.0008732902558057627],
    [-0.0008732902558057627, 0.06229470491414441]
])

In [55]:
cov_beta_hat[0][0] ** 0.5

0.05528576575683988

In [56]:
cov_beta_hat[1][1] ** 0.5

0.24958907210481876

and these match the results in the `statsmodels` summary